# Lecture 4: Password CPA Attack - Visualizations

## Exercise 1

In [10]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper

output_notebook()

Loading BokehJS ...

In [2]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [3]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [5]:
import securec
from securec import util
scope, target = util.init()

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


In [6]:
securec.util.compile_and_flash('./4_password_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2041 bytes
✓


In [7]:
import numpy as np

scope.default_setup()

def capture(attempt, samples=500):
    scope.adc.samples = samples
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    scope.arm()
    target.simpleserial_write(0x01, attempt + b'\x00' * (10 - len(attempt)))
    result = target.simpleserial_read(0x01, 1)
    return np.array(util.capture()), not bool(result[0])


In [8]:
import random
import tqdm
import tqdm.notebook

trace_samples = 500
trace_nums = 1000

traces = []
attempts = []
for _ in tqdm.notebook.tqdm(range(trace_nums)):
    attempt = bytes([random.randint(0, 255) for _ in range(10)])
    traces.append(capture(attempt, samples=trace_samples)[0])
    attempts.append(attempt)
traces = np.array(traces)
attempts = np.array([list(a) for a in attempts])


  0%|          | 0/1000 [00:00<?, ?it/s]

ERROR:ChipWhisperer Target:Device did not ack


In [9]:
colormap = LinearColorMapper(
    palette='Viridis256',
    low=0,
    high=1,
)

chars = list('abcdefghijklmnopqrstuvwxyz')

for idx in range(8):
    pearsons = [abs(pearson_pointwise(traces, hw_vec(attempts[:, idx] ^ ord(i)))) for i in chars]
    df = pd.DataFrame(pearsons, index=chars)
    df = df.stack().reset_index()
    df.columns=['char', 'point', 'value']


    p = figure(
        y_range=chars, 
        height=400,
        sizing_mode='stretch_width',
        tooltips=[
            ("char", "@char"),
            ("corr", "@value"),
        ],
        title=f'Correlations for guessing position {idx}'
    )

    p.rect(
        width=1,
        height=1,
        source=df,
        x='point',
        y='char',
        fill_color={'field': 'value', 'transform': colormap},
        line_color=None,
    )
    show(p)

/tmp/ipykernel_364927/3214675746.py:6: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (


In [11]:
import math

def pearson_pointwise_multi(traces, intermediates):
    (n, t) = traces.shape
    (_, m) = intermediates.shape

    d_traces = traces - np.einsum('nt->t', traces, dtype='float64', optimize='optimal') / np.double(n)
    d_intermediates = intermediates - np.einsum('nm->m', intermediates, dtype='float64', optimize='optimal') / np.double(n)
    
    tmp1 = np.einsum('nm,nm->m', d_intermediates, d_intermediates, optimize='optimal')
    tmp2 = np.einsum('nt,nt->t', d_traces, d_traces, optimize='optimal')
    tmp = np.einsum('m,t->mt', tmp1, tmp2, optimize='optimal')
    denominator = np.sqrt(tmp)
    numerator = np.einsum('nm,nt->mt', d_intermediates, d_traces, optimize='optimal')

    return np.nan_to_num(numerator / denominator)

def plot_correlation_vs_traces(
    traces,
    textins,
    password_index=0,
    trylist='abcdefghijklmnopqrstuvwxyz0123456789',
    plotpoints=500,
):
    # Compute data
    plotpoints = min(plotpoints, len(traces))
    data = np.zeros((len(trylist), plotpoints))
    intermediates = np.array([[hw(attempt[password_index] ^ ord(guess)) for guess in trylist] for attempt in textins])
    for i in range(0, plotpoints):
        j = math.ceil(i / plotpoints * len(traces))
        data[:, i] = np.max(np.abs(pearson_pointwise_multi(traces[:j, :], intermediates[:j, :])), axis=1)
    
    source = {
        'xs': len(data) * [list(range(0, len(traces), math.ceil(len(traces) / plotpoints)))],
        'ys': [corr for corr in data],
        'legend': list(trylist),
        'color': math.ceil(len(data) / 20) * bokeh.palettes.Category20_20,
    }
    # Create figure
    p = figure(sizing_mode='stretch_width', height=300, tooltips=[('char', '@legend'), ('corrleation', '$y')])
    p.multi_line(xs='xs', ys='ys', color='color', source=source)
    return p

In [12]:
show(plot_correlation_vs_traces(traces, attempts))

/tmp/ipykernel_364927/2800307057.py:7: RuntimeWarning: invalid value encountered in true_divide
  d_traces = traces - np.einsum('nt->t', traces, dtype='float64', optimize='optimal') / np.double(n)
/tmp/ipykernel_364927/2800307057.py:8: RuntimeWarning: invalid value encountered in true_divide
  d_intermediates = intermediates - np.einsum('nm->m', intermediates, dtype='float64', optimize='optimal') / np.double(n)
/tmp/ipykernel_364927/2800307057.py:16: RuntimeWarning: invalid value encountered in true_divide
  return np.nan_to_num(numerator / denominator)
